[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/finance-data-analysis/blob/main/3.3%20%EB%84%A4%EC%9D%B4%EB%B2%84%EA%B8%88%EC%9C%B5%20%EA%B0%9C%EB%B3%84%EC%A2%85%EB%AA%A9%20%EC%88%98%EC%A7%91-input.ipynb)


## 네이버 금융 개별종목 수집
* FinanceDataReader를 통해 수집했던 데이터를 네이버 증권 웹 페이지를 통해 직접 수집합니다.


### Keyword

* html 파일 읽어오기
    * pd.read_html(url, encoding="cp949")

* 결측 데이터 제거하기(axis 0:행, 1:열)
    * table[0].dropna()

* 데이터 프레임 합치기
    * pd.concat([df1, df2, df3])

* 중복데이터 제거
    * df.drop_duplicates()

* 과학적 기수법
    * 1.210000e+02 => 121

* 날짜 column의 첫 row값 확인
    * date = df.iloc[0]["날짜"]

* 파일로 저장하기 
    * df.to_csv(file_name, index=False)

* 파일 읽어오기
    * pd.read_csv(file_name)

## 수집할 페이지 보기

* 네이버 금융 국내증시 : https://finance.naver.com/sise/
* 2020년 주요 상장종목
    * 빅히트 : https://finance.naver.com/item/main.nhn?code=352820
    * 카카오게임즈 : https://finance.naver.com/item/main.nhn?code=293490
    * SK바이오팜 : https://finance.naver.com/item/main.nhn?code=326030

## 라이브러리 로드

In [1]:
# 라이브러리 로드
import pandas as pd

## 수집할 URL 정하기

In [4]:
# 종목번호와 상장사 이름을 item_code와 item_name으로 설정
# item_code = "352820"
# item_name = "빅히트"

item_code = "326030"
item_name = "SK바이오팜"
page_no = 1

# 종목 URL 만들기
url = f"https://finance.naver.com/item/sise_day.nhn?code={item_code}&page={page_no}"
url

'https://finance.naver.com/item/sise_day.nhn?code=326030&page=1'

## requests를 통한 HTTP 요청
* [Requests: HTTP for Humans™ — Requests documentation](https://requests.readthedocs.io/en/master/)
* [Quickstart — Requests documentation # custom-headers](https://requests.readthedocs.io/en/latest/user/quickstart/#custom-headers)

In [ ]:
import requests

headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}

response = requests.get(url, headers=headers)
response.text

## BeautifulSoup 을 통한 table 태그 찾기

* [Beautiful Soup Documentation — Beautiful Soup 4.9.0 documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

In [7]:
from bs4 import BeautifulSoup as bs

html = bs(response.text, "lxml")
table = html.select("table")
len(table)

2

In [8]:
type(table)

bs4.element.ResultSet

## pandas 코드 한 줄로 데이터 수집하기

In [9]:
# read_html을 이용하여 url의 page내의 값을 DataFrame으로 받아옵니다.
# cp949는 한글 인코딩을 위해 사용합니다. 기본 인코딩 설정은 utf-8 이며, 
# 네이버의 일별 시세는 cp949 인코딩으로 불러올 수 있습니다.
# 데이터를 로드 했을 때 한글 인코딩이 깨진다면 대부분 cp949 로 불러올 수 있습니다.
table = pd.read_html(str(table))

In [10]:
# table[0]와 table[1]을 확인하여 보면 table[0]에 필요한 데이터들이 있습니다.
table[1]

,0,1,2,3,4,5,6,7,8,9,10,11
0,1,2,3,4,5,6,7,8,9,10,다음,맨뒤


In [11]:
table[0]

,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022.06.22,70600.0,100.0,70200.0,72100.0,69700.0,99300.0
2,2022.06.21,70500.0,3100.0,67400.0,70800.0,67300.0,117753.0
3,2022.06.20,67400.0,2200.0,69800.0,70600.0,66300.0,154629.0
4,2022.06.17,69600.0,1000.0,67200.0,70900.0,67200.0,206494.0
5,2022.06.16,70600.0,500.0,72200.0,72900.0,70200.0,122069.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2022.06.15,71100.0,3400.0,73600.0,74700.0,70900.0,212995.0


In [12]:
# dropna를 통해 결측치가 들어있는 row를 제거합니다.
temp = table[0].dropna()
temp

,날짜,종가,전일비,시가,고가,저가,거래량
1,2022.06.22,70600.0,100.0,70200.0,72100.0,69700.0,99300.0
2,2022.06.21,70500.0,3100.0,67400.0,70800.0,67300.0,117753.0
3,2022.06.20,67400.0,2200.0,69800.0,70600.0,66300.0,154629.0
4,2022.06.17,69600.0,1000.0,67200.0,70900.0,67200.0,206494.0
5,2022.06.16,70600.0,500.0,72200.0,72900.0,70200.0,122069.0
9,2022.06.15,71100.0,3400.0,73600.0,74700.0,70900.0,212995.0
10,2022.06.14,74500.0,1900.0,73700.0,76300.0,73700.0,150870.0
11,2022.06.13,76400.0,4600.0,79700.0,80000.0,76300.0,159796.0
12,2022.06.10,81000.0,1400.0,82100.0,82600.0,80500.0,103312.0
13,2022.06.09,82400.0,100.0,82500.0,82500.0,81400.0,107945.0


## 페이지별 데이터 수집 함수 만들기

In [13]:
# 종목 번호를 이용해 page에 따라 데이터를 읽어오는 함수
# """ 는 이 두개 사이의 행들은 주석 처리되며, 함수의 docstring 으로 사용됩니다.

def get_day_list(item_code, page_no):
    """
    일자별 시세를 페이지별로 수집
    """ 
    url = f"https://finance.naver.com/item/sise_day.nhn?code={item_code}&page={page_no}"
    
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}

    response = requests.get(url, headers=headers)
    html = bs(response.text, "lxml")
    table = html.select("table")
    table = pd.read_html(str(table))
    temp = table[0].dropna()
    return temp

In [14]:
# 함수가 잘 만들어졌는지 확인하기
get_day_list(item_code, 1)

,날짜,종가,전일비,시가,고가,저가,거래량
1,2022.06.22,70400.0,100.0,70200.0,72100.0,69700.0,100352.0
2,2022.06.21,70500.0,3100.0,67400.0,70800.0,67300.0,117753.0
3,2022.06.20,67400.0,2200.0,69800.0,70600.0,66300.0,154629.0
4,2022.06.17,69600.0,1000.0,67200.0,70900.0,67200.0,206494.0
5,2022.06.16,70600.0,500.0,72200.0,72900.0,70200.0,122069.0
9,2022.06.15,71100.0,3400.0,73600.0,74700.0,70900.0,212995.0
10,2022.06.14,74500.0,1900.0,73700.0,76300.0,73700.0,150870.0
11,2022.06.13,76400.0,4600.0,79700.0,80000.0,76300.0,159796.0
12,2022.06.10,81000.0,1400.0,82100.0,82600.0,80500.0,103312.0
13,2022.06.09,82400.0,100.0,82500.0,82500.0,81400.0,107945.0


## 반복문을 통한 전체 일자 데이터 수집하기
* (주의) 기간이 긴 데이터를 수집할때는 서버에 부담을 주지 않기 위해 time.sleep()값을 주세요.

In [15]:
import time
# web page 시작번호
page_no = 1
# 데이터를 저장할 빈 변수 선언
item_list = []
while True:
    print(page_no)
    temp = get_day_list(item_code, page_no)
    item_list.append(temp)
    
    page_no = page_no + 1
    time.sleep(0.1)
    
    #if temp.shape[0] < 10:
    if page_no > 10:
        break;

1
2
3
4
5
6
7
8
9
10


In [16]:
len(item_list)

10

## 수집한 데이터 하나의 데이터프레임으로 합치기

<img src="https://pandas.pydata.org/docs/_images/merging_concat_basic.png">

* [Merge, join, concatenate and compare documentation](https://pandas.pydata.org/docs/user_guide/merging.html#merge-join-concatenate-and-compare)

In [17]:
# DataFrame들이 list의 형태로 저장된 것을 concat을 이용하여 합치면서 하나의 DataFrame으로 만듭니다.
df = pd.concat(item_list)

<img src="https://pandas.pydata.org/docs/_images/08_concat_row.svg">

In [18]:
# head와 tail로 데이터의 일부를 가져와서 봅니다.
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2022.06.22,70400.0,100.0,70200.0,72100.0,69700.0,100775.0
2,2022.06.21,70500.0,3100.0,67400.0,70800.0,67300.0,117753.0
3,2022.06.20,67400.0,2200.0,69800.0,70600.0,66300.0,154629.0
4,2022.06.17,69600.0,1000.0,67200.0,70900.0,67200.0,206494.0
5,2022.06.16,70600.0,500.0,72200.0,72900.0,70200.0,122069.0


In [19]:
df.tail()

,날짜,종가,전일비,시가,고가,저가,거래량
9,2022.01.28,75800.0,2600.0,73400.0,76500.0,73100.0,189184.0
10,2022.01.27,73200.0,5100.0,78300.0,78600.0,73000.0,354564.0
11,2022.01.26,78300.0,1300.0,79100.0,80900.0,78000.0,154140.0
12,2022.01.25,79600.0,3500.0,83200.0,83600.0,79100.0,268615.0
13,2022.01.24,83100.0,2100.0,84800.0,85500.0,82800.0,152840.0


## 데이터프레임에 종목코드와 종목명을 추가하기
* 파생변수 만들기

In [20]:
# '종목코드'와 '종목명' column을 추가하면서 각각 item_code와 item_name 값을 입력합니다.
df["종목코드"] = item_code
df["종목명"] = item_name
df

,날짜,종가,전일비,시가,고가,저가,거래량,종목코드,종목명
1,2022.06.22,70400.0,100.0,70200.0,72100.0,69700.0,100775.0,326030,SK바이오팜
2,2022.06.21,70500.0,3100.0,67400.0,70800.0,67300.0,117753.0,326030,SK바이오팜
3,2022.06.20,67400.0,2200.0,69800.0,70600.0,66300.0,154629.0,326030,SK바이오팜
4,2022.06.17,69600.0,1000.0,67200.0,70900.0,67200.0,206494.0,326030,SK바이오팜
5,2022.06.16,70600.0,500.0,72200.0,72900.0,70200.0,122069.0,326030,SK바이오팜
...,...,...,...,...,...,...,...,...,...
9,2022.01.28,75800.0,2600.0,73400.0,76500.0,73100.0,189184.0,326030,SK바이오팜
10,2022.01.27,73200.0,5100.0,78300.0,78600.0,73000.0,354564.0,326030,SK바이오팜
11,2022.01.26,78300.0,1300.0,79100.0,80900.0,78000.0,154140.0,326030,SK바이오팜
12,2022.01.25,79600.0,3500.0,83200.0,83600.0,79100.0,268615.0,326030,SK바이오팜


## 컬럼 순서 변경하기

In [21]:
# DataFrame에서 column 들의 이름을 순서를 조정하여 column순서를 변경할 수 있습니다.
df.columns

Index(['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량', '종목코드', '종목명'], dtype='object')

In [22]:
cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
df = df[cols]

## 중복데이터 제거하기
* drop_duplicates 를 통해 중복된 데이터가 있다면 제거합니다.

In [23]:
df.shape

(100, 9)

In [24]:
# drop_duplicates : row들 끼리 data를 비교하여 같은 값이 있으면 row중 하나를 삭제
# df.shape를 전/후로 출력하여 삭제된 row가 있는지 확인
df = df.drop_duplicates()
df.shape

(100, 9)

## 기술통계값 구하기

In [25]:
# describe는 기본적으로 수치데이터에 대한 기술통계값을 구하게 됩니다.
df.describe()

,종가,전일비,시가,고가,저가,거래량
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,84639.000000,1442.000000,84584.000000,85898.000000,83433.000000,142817.350000
std,5828.281709,1272.250849,5753.693939,5452.883215,5869.198059,98415.212639
min,67400.000000,0.000000,67200.000000,70600.000000,66300.000000,50318.000000
25%,80650.000000,475.000000,81150.000000,82575.000000,79175.000000,90578.250000
50%,86600.000000,1100.000000,86500.000000,87800.000000,85400.000000,113243.500000
75%,88725.000000,1900.000000,88425.000000,89325.000000,87525.000000,158365.750000
max,93100.000000,6900.000000,92800.000000,94300.000000,91400.000000,715346.000000


In [42]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df.describe()

,종가,전일비,시가,고가,저가,거래량
count,100.00,100.00,100.00,100.00,100.00,100.00
mean,84639.00,1442.00,84584.00,85898.00,83433.00,142817.35
std,5828.28,1272.25,5753.69,5452.88,5869.20,98415.21
min,67400.00,0.00,67200.00,70600.00,66300.00,50318.00
25%,80650.00,475.00,81150.00,82575.00,79175.00,90578.25
50%,86600.00,1100.00,86500.00,87800.00,85400.00,113243.50
75%,88725.00,1900.00,88425.00,89325.00,87525.00,158365.75
max,93100.00,6900.00,92800.00,94300.00,91400.00,715346.00


* 참고 : E notation
[과학적 기수법 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EA%B3%BC%ED%95%99%EC%A0%81_%EA%B8%B0%EC%88%98%EB%B2%95)

과학적 기수법, 과학적 표기법(scientific notation, scientific form, standard index form, standard form)은 너무 크거나 너무 작은 숫자들을 십진법으로 편하게 작성하여 표현하는 방법이다. 과학자, 수학자, 공학자들이 공통적으로 사용하는데, 부분적인 이유는 특정한 산술을 단순화시켜 주기 때문이다. 과학 계산기에서는 "SCI" 디스플레이 모드라는 이름으로 알려져 있다.

In [26]:
# 거래량의 과학적 기수법 읽기
1.000000e+02

100.0

In [27]:
4.701523e+05

470152.3

In [28]:
4.701523 * (10 ** 5)

470152.3

## 최근 날짜 구해서 파일명 만들기

In [32]:
# 날짜 column의 첫 row값 확인
date = df.iloc[0]["날짜"]
date

'2022.06.22'

In [33]:
# 종목명, 종목코드, 날짜를 이름으로 하는 csv 파일명 만들기
file_name = f"{item_name}_{item_code}_{date}.csv"
file_name

'SK바이오팜_326030_2022.06.22.csv'

## 파일로 저장하기

In [34]:
# 파일로 저장하기 
# index=False 로 데이터프레임의 기본 index 는 저장하지 않도록 합니다.
df.to_csv(file_name, index=False)

In [35]:
# 제대로 저장되었는지 파일을 읽어서 확인합니다.
pd.read_csv(file_name)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
0,326030,SK바이오팜,2022.06.22,70400.0,100.0,70200.0,72100.0,69700.0,100775.0
1,326030,SK바이오팜,2022.06.21,70500.0,3100.0,67400.0,70800.0,67300.0,117753.0
2,326030,SK바이오팜,2022.06.20,67400.0,2200.0,69800.0,70600.0,66300.0,154629.0
3,326030,SK바이오팜,2022.06.17,69600.0,1000.0,67200.0,70900.0,67200.0,206494.0
4,326030,SK바이오팜,2022.06.16,70600.0,500.0,72200.0,72900.0,70200.0,122069.0
...,...,...,...,...,...,...,...,...,...
95,326030,SK바이오팜,2022.01.28,75800.0,2600.0,73400.0,76500.0,73100.0,189184.0
96,326030,SK바이오팜,2022.01.27,73200.0,5100.0,78300.0,78600.0,73000.0,354564.0
97,326030,SK바이오팜,2022.01.26,78300.0,1300.0,79100.0,80900.0,78000.0,154140.0
98,326030,SK바이오팜,2022.01.25,79600.0,3500.0,83200.0,83600.0,79100.0,268615.0


## 전체 과정을 하나의 함수로 만들기

In [37]:
def get_item_list(item_code, item_name):
    """
    일별 시세를 수집하는 함수
    """
    # web page 시작번호
    page_no = 1
    # 데이터를 저장할 빈 변수 선언
    item_list = []

    while True:
        temp = get_day_list(item_code, page_no)
        item_list.append(temp)

        page_no = page_no + 1
        time.sleep(0.1)

        #if temp.shape[0] < 10:
        if page_no > 10:
            break
            
    df = pd.concat(item_list)
    df["종목코드"] = item_code
    df["종목명"] = item_name
    cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
    df = df[cols]

    return df

In [38]:
item_code = "352820"
item_name = "하이브"
# 빅히트의 사명이 하이브로 변경되었습니다.

# item_code = "326030"
# item_name = "SK바이오팜"

get_item_list(item_code, item_name)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
1,352820,하이브,2022.06.22,141000.0,8500.0,152000.0,152500.0,140000.0,406203.0
2,352820,하이브,2022.06.21,149500.0,0.0,150000.0,151500.0,139500.0,506620.0
3,352820,하이브,2022.06.20,149500.0,1000.0,151500.0,152000.0,145500.0,489287.0
4,352820,하이브,2022.06.17,148500.0,500.0,141000.0,153000.0,140500.0,879832.0
5,352820,하이브,2022.06.16,148000.0,3000.0,148000.0,158000.0,139000.0,2713956.0
...,...,...,...,...,...,...,...,...,...
9,352820,하이브,2022.01.28,237000.0,4500.0,244500.0,244500.0,229500.0,408005.0
10,352820,하이브,2022.01.27,241500.0,10000.0,250000.0,255000.0,241500.0,309387.0
11,352820,하이브,2022.01.26,251500.0,19500.0,271500.0,272500.0,251000.0,430078.0
12,352820,하이브,2022.01.25,271000.0,6000.0,280000.0,281500.0,269000.0,180016.0


## 내가 만든 함수의 기능과 소스코드가 궁금하다면?

In [39]:
# 도움말 보기
get_item_list?


In [40]:
# 소스코드 보기
get_item_list??
